# Part 3 - Genres Analysis

In [0]:
spark

sc = spark.sparkContext

In [0]:
filepath = "/Users/c.maulard@gmail.com/SteamProject/Steam_data_dataset_zoomed_1.json"

df = spark.read.json(filepath)

In [0]:
from pyspark.sql import functions as F # This will load the class where spark sql functions are contained
from pyspark.sql import Row # this will let us manipulate rows with spark sql
from functools import reduce
from pyspark.sql.functions import when

In [0]:
df.printSchema()

root
 |-- appid: long (nullable = true)
 |-- categories: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- ccu: long (nullable = true)
 |-- developer: string (nullable = true)
 |-- discount: string (nullable = true)
 |-- genre: string (nullable = true)
 |-- header_image: string (nullable = true)
 |-- initialprice: string (nullable = true)
 |-- languages: string (nullable = true)
 |-- name: string (nullable = true)
 |-- negative: long (nullable = true)
 |-- owners: string (nullable = true)
 |-- platforms: struct (nullable = true)
 |    |-- linux: boolean (nullable = true)
 |    |-- mac: boolean (nullable = true)
 |    |-- windows: boolean (nullable = true)
 |-- positive: long (nullable = true)
 |-- price: string (nullable = true)
 |-- publisher: string (nullable = true)
 |-- release_date: string (nullable = true)
 |-- required_age: string (nullable = true)
 |-- short_description: string (nullable = true)
 |-- tags: struct (nullable = true)
 |    |-- 1980s: lon

In [0]:
df.select('genre').limit(10).display()

genre
Action
"Action, Adventure, Indie"
"Adventure, Indie, RPG, Strategy"
"Action, Indie, Simulation, Strategy"
"Action, Casual, Indie, Simulation"
"Action, Adventure, Indie, RPG"
"Adventure, Indie, RPG, Strategy"
"Action, Adventure, Casual, Free to Play, Massively Multiplayer"
"Casual, Indie"
"Indie, RPG"


In [0]:
col_to_drop = ['tags', 'header_image', 'platforms', 'short_description', 'type', 'website']

def dropCol(df, col_name):
  return df.drop(*col_name)

df_genre = dropCol(df, col_to_drop)
df_genre.limit(5).toPandas()

,appid,categories,ccu,developer,discount,genre,initialprice,languages,name,negative,owners,positive,price,publisher,release_date,required_age
0,10,"[Multi-player, Valve Anti-Cheat enabled, Onlin...",13990,Valve,0,Action,999,"English, French, German, Italian, Spanish - Sp...",Counter-Strike,5199,"10,000,000 .. 20,000,000",201215,999,Valve,2000/11/1,0
1,1000000,"[Single-player, Partial Controller Support, St...",0,IndigoBlue Game Studio,0,"Action, Adventure, Indie",999,"English, Korean, Simplified Chinese",ASCENXION,5,"0 .. 20,000",27,999,PsychoFlux Entertainment,2021/05/14,0
2,1000010,"[Single-player, Partial Controller Support, St...",99,NEXT Studios,70,"Adventure, Indie, RPG, Strategy",1999,"Simplified Chinese, English, Japanese, Traditi...",Crown Trick,646,"200,000 .. 500,000",4032,599,"Team17, NEXT Studios",2020/10/16,0
3,1000030,"[Multi-player, Single-player, Co-op, Steam Ach...",76,Vertigo Gaming Inc.,0,"Action, Indie, Simulation, Strategy",1999,English,"Cook, Serve, Delicious! 3?!",115,"100,000 .. 200,000",1575,1999,Vertigo Gaming Inc.,2020/10/14,0
4,1000040,[Single-player],0,DoubleC Games,0,"Action, Casual, Indie, Simulation",199,Simplified Chinese,细胞战争,1,"0 .. 20,000",0,199,DoubleC Games,2019/03/30,0


In [0]:
df_genres = df_genre.withColumn("genre",F.split(df_genre["genre"], ", "))

df_genres.limit(5).display()

appid,categories,ccu,developer,discount,genre,initialprice,languages,name,negative,owners,positive,price,publisher,release_date,required_age
10,"List(Multi-player, Valve Anti-Cheat enabled, Online PvP, Shared/Split Screen PvP, PvP)",13990,Valve,0,List(Action),999,"English, French, German, Italian, Spanish - Spain, Simplified Chinese, Traditional Chinese, Korean",Counter-Strike,5199,"10,000,000 .. 20,000,000",201215,999,Valve,2000/11/1,0
1000000,"List(Single-player, Partial Controller Support, Steam Achievements, Steam Cloud)",0,IndigoBlue Game Studio,0,"List(Action, Adventure, Indie)",999,"English, Korean, Simplified Chinese",ASCENXION,5,"0 .. 20,000",27,999,PsychoFlux Entertainment,2021/05/14,0
1000010,"List(Single-player, Partial Controller Support, Steam Achievements, Steam Cloud, Steam Trading Cards)",99,NEXT Studios,70,"List(Adventure, Indie, RPG, Strategy)",1999,"Simplified Chinese, English, Japanese, Traditional Chinese, French, German, Spanish - Spain, Russian, Portuguese - Brazil",Crown Trick,646,"200,000 .. 500,000",4032,599,"Team17, NEXT Studios",2020/10/16,0
1000030,"List(Multi-player, Single-player, Co-op, Steam Achievements, Steam Cloud, Shared/Split Screen, Full controller support, Steam Trading Cards, Shared/Split Screen Co-op, Remote Play on Phone, Remote Play on Tablet, Remote Play on TV, Remote Play Together)",76,Vertigo Gaming Inc.,0,"List(Action, Indie, Simulation, Strategy)",1999,English,"Cook, Serve, Delicious! 3?!",115,"100,000 .. 200,000",1575,1999,Vertigo Gaming Inc.,2020/10/14,0
1000040,List(Single-player),0,DoubleC Games,0,"List(Action, Casual, Indie, Simulation)",199,Simplified Chinese,细胞战争,1,"0 .. 20,000",0,199,DoubleC Games,2019/03/30,0


### What are the most represented genres?
### 


In [0]:
df_genres_count = df_genres.select('name', F.explode('genre').alias('genre'))
display(df_genres_count.limit(10))

name,genre
Counter-Strike,Action
ASCENXION,Action
ASCENXION,Adventure
ASCENXION,Indie
Crown Trick,Adventure
Crown Trick,Indie
Crown Trick,RPG
Crown Trick,Strategy
"Cook, Serve, Delicious! 3?!",Action
"Cook, Serve, Delicious! 3?!",Indie


In [0]:
game_count_genres = df_genres_count.select('genre').distinct().count()

display(game_count_genres)

29

In [0]:
df_genres_count\
    .groupBy("genre")\
    .count().withColumnRenamed("count", "game_per_genre")\
    .orderBy(F.desc("game_per_genre"))\
    .limit(10).display()

genre,game_per_genre
Indie,39681
Action,23759
Casual,22086
Adventure,21431
Strategy,10895
Simulation,10836
RPG,9534
Early Access,6145
Free to Play,3393
Sports,2666


Databricks visualization. Run in Databricks to view.

Les jeux indépendants sont les plus représentés

### Are there any genres that have a better positive/negative review ratio?

In [0]:
df_rated_genres = df_genres.select('name', 'positive', 'negative', F.explode('genre').alias('genre'))

In [0]:
rated_genres = df_rated_genres.withColumn('rate_ratio', F.col('positive')/F.col('negative'))
rated_genres.groupBy('genre').sum('rate_ratio')\
    .withColumnRenamed("sum(rate_ratio)", "rate_ratio_per_genre")\
    .orderBy(F.desc('rate_ratio_per_genre')).limit(10).display()

genre,rate_ratio_per_genre
Indie,197628.52578864107
Adventure,115147.83941327479
Casual,107982.78879894866
Action,107068.0707142928
Simulation,47666.7064755077
RPG,47578.155694970534
Strategy,45011.068672994494
Early Access,23576.8836452649
Free to Play,17550.37769389576
Sports,10120.119311420696


Databricks visualization. Run in Databricks to view.

Les jeux indépendants ont le meilleur ratio notes +/ notes - 

### Do some publishers have favorite genres?
### 


In [0]:
df_publisher_genres = df_genres.select('publisher', F.explode('genre').alias('genre'))

In [0]:
df_genre_publisher_count = df_publisher_genres.groupBy("publisher", "genre").count()

df_genre_publisher_count.limit(10).display()

publisher,genre,count
Emilios Manolidis,Indie,3
Polygone Games,Adventure,1
Rollman team,Indie,1
EyeContact Games,Adventure,1
"Fair Games Studio, Pleasant Rain Ltd",Adventure,2
Rising Sun Interactive,Indie,3
FG,Indie,2
Secret Forest Games,Adventure,1
Innovative Underdogs,Adventure,1
Glass Bubble Software,Indie,1


In [0]:
from pyspark.sql.window import Window

w = Window.partitionBy("publisher").orderBy(F.desc("count"))

df_ranked = df_genre_publisher_count.withColumn("rank", F.rank().over(w))
df_top_genre_per_publisher = df_ranked.filter((F.col("rank") == 1) &  (F.col("count") >= 10)).drop("rank")

df_top_genre_per_publisher.display()

publisher,genre,count
505 Games,Action,28
8Floor,Casual,41
8floor,Casual,202
Activision,Action,28
Aerosoft GmbH,Simulation,33
Akupara Games,Indie,15
Amaterasu Software,Indie,10
Annapurna Interactive,Adventure,21
Application Systems Heidelberg,Indie,11
"Arcen Games, LLC",Indie,11


In [0]:
df_top_genre_per_publisher.groupBy('genre').count()\
    .orderBy(F.desc('count')).limit(10).display()

genre,count
Indie,253
Casual,89
Action,77
Adventure,59
Simulation,24
RPG,23
Strategy,20
Sports,5
Video Production,2
Racing,2


Databricks visualization. Run in Databricks to view.

Le meilleur genre pour les editeurs sont les genre indépendants

### What are the most lucrative genres?
### 


In [0]:
from pyspark.sql.functions import regexp_replace, expr

df_price_genres = df_genres.select('price', 'owners', F.explode('genre').alias('genre'))
df_price_genres = df_price_genres.withColumn("price", (F.col("price") / 100).cast("decimal(10,2)"))
df_price_genres = df_price_genres.withColumn("owners", regexp_replace(df_price_genres["owners"], ",", "")) \
       .withColumn("owners", expr("split(owners, ' \\.. ')"))

df_price_genres = df_price_genres.withColumn("owners", expr("transform(owners, x -> cast(x as int))"))
df_price_genres = df_price_genres.withColumn(
    "owners_avg",
    expr("aggregate(owners, 0, (acc, x) -> acc + x) / size(owners)")
)

df_price_genres.limit(10).display()

price,owners,genre,owners_avg
9.99,"List(10000000, 20000000)",Action,1.5E7
9.99,"List(0, 20000)",Action,10000.0
9.99,"List(0, 20000)",Adventure,10000.0
9.99,"List(0, 20000)",Indie,10000.0
5.99,"List(200000, 500000)",Adventure,350000.0
5.99,"List(200000, 500000)",Indie,350000.0
5.99,"List(200000, 500000)",RPG,350000.0
5.99,"List(200000, 500000)",Strategy,350000.0
19.99,"List(100000, 200000)",Action,150000.0
19.99,"List(100000, 200000)",Indie,150000.0


In [0]:
df_price_genres = df_price_genres.withColumn("financial_return", F.col('price') * F.col('owners_avg'))

df_price_genres.limit(10).display()

price,owners,genre,owners_avg,financial_return
9.99,"List(10000000, 20000000)",Action,1.5E7,1.4985E8
9.99,"List(0, 20000)",Action,10000.0,99900.0
9.99,"List(0, 20000)",Adventure,10000.0,99900.0
9.99,"List(0, 20000)",Indie,10000.0,99900.0
5.99,"List(200000, 500000)",Adventure,350000.0,2096500.0
5.99,"List(200000, 500000)",Indie,350000.0,2096500.0
5.99,"List(200000, 500000)",RPG,350000.0,2096500.0
5.99,"List(200000, 500000)",Strategy,350000.0,2096500.0
19.99,"List(100000, 200000)",Action,150000.0,2998499.9999999995
19.99,"List(100000, 200000)",Indie,150000.0,2998499.9999999995


In [0]:
lucrative_genre = df_price_genres.groupby('genre').sum('financial_return')\
    .withColumnRenamed("sum(financial_return)", "financial_return_per_genre")\
    .orderBy(F.desc('financial_return_per_genre'))

lucrative_genre.limit(10).display()

genre,financial_return_per_genre
Action,5.87564541E10
Adventure,3.724573845E10
Indie,3.23465771E10
RPG,2.71731431E10
Strategy,2.015004105E10
Simulation,1.876974955E10
Casual,8.08095665E9
Massively Multiplayer,5.93015775E9
Early Access,5.45866145E9
Sports,3.14989735E9


Databricks visualization. Run in Databricks to view.

Les jeux d'action génèrent le plus de revenus